In [1]:
import benepar, spacy
benepar.download('benepar_en3')

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
doc = nlp('The time for action is now. It is never too late to do something.')
sent = list(doc.sents)[0]

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/kostadindev/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [2]:
#list(sent._.children)[3]
#print(list(sent._.children)[2].label)
# for i in range(len(list(sent._.children))):
#     print(list(sent._.children[i]), list(sent._._labels)[i])
# list(sent._.children)
import pandas as pd
from pprint import pprint as pprint
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance
sentences = pd.read_csv("sentences.csv")
sentences = sentences.to_numpy().squeeze()
signs = [',', ':', ';', '.', '!', '?']
recursive = True
max_similarity= 5

In [84]:
def pd_print_entry(df, index):
    entry = df.iloc[index]
    print("Sentence: ")
    print(entry['sentence'])
    print("Questions: ")
    print(entry['questions'])
    print('Answers')
    #print(entry['answers'])
    for answer in entry['answers']:
        print(answer)
    print('Removed')
    #print(entry['answers'])
    for removed in entry['removed']:
        print(removed)
    print('Phrases: ')
    pprint(entry['phrases'])

In [85]:
def get_phrases(sent, recursive = False):
    children = list(sent._.children)
    #VP, NP, SBAR, ADVP, PP = [], [], [], [], []
    phrases = {}
    for i in range(len(children)):
        if str(children[i]) not in signs:
            try:
                label = children[i]._.labels[0]
            except:
                label= "stop"
            # print(children[i], label)
            # print(children[i], children[i]._.labels)
            # generate_questions(children[i])
            if label != 'stop':
                if label not in phrases.keys():
                    phrases[label] = []
                phrases[label].append(children[i])
                new_phrases= get_phrases(children[i], recursive= recursive)
                for key in new_phrases.keys():
                    if key not in phrases.keys():
                        phrases[key] = []
                    phrases[key].extend(new_phrases[key])
    return phrases

In [86]:
def get_QA(phrases):
    questions, answers = [], []
    if 'SBAR' in phrases.keys():
        for sbar in phrases['SBAR']:
            questions.append(f"What {str(sbar).lower()}?")
            if 'NP' in phrases.keys():
                for np in phrases['NP']:
                    if 'VP' in phrases.keys():
                        for vp in phrases['VP']:
                            answers.append(f'<np>{np}<np> <vp>{vp}<vp>')
    return questions, answers

In [87]:
def edit_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [96]:
def filter_edit_distance(words):
    if len(words) == 0:
        return []
    filtered_words = []
    words = np.asarray(words) #So that indexing with a list will work
    lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

    affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(lev_similarity)
    for cluster_id in np.unique(affprop.labels_):
        #exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
        cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
        filtered_words.append(min(cluster, key = lambda x: len(x)))
    return filtered_words

In [97]:
def filter_not_in_question(answers, question):
    filtered_answers = []
    for answer in answers:
        vp = answer.split('<vp>')[1]
        np = answer.split('<np>')[1]
        if (vp not in question or len(vp)<10) and (np not in question or len(np)<10):
            filtered_answers.append(answer)
    return filtered_answers

In [114]:
def parse_sentence(sentence):
    sent = list(nlp(sentence).sents)[0]
    phrases = get_phrases(sent, recursive = recursive)
    questions, answers = get_QA(phrases)
    qa_pairs = []
    filtered_answers = filter_edit_distance(answers)
    # for question in questions:
    #     filtered_answers = answers #filter_not_in_question(answers, question)
    #     qa_pairs.append({'question':question, 'answers':filtered_answers})
    return [str(sent), questions, filtered_answers, [answer for answer in answers if answer not in filtered_answers], phrases]

In [ ]:
if __name__ == '__main__':
    entries = []
    for idx, sentence in enumerate(sentences):
        entries.append(parse_sentence(sentence))
        print(f"Finished {idx}/{len(sentences)}")
    df = pd.DataFrame(entries, columns =['sentence', 'questions', 'answers','removed' 'phrases'])
    df.to_csv('parsed_sentences.csv')

/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


Finished 0/23


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


Finished 1/23
Finished 2/23


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


...

Finished 10/23


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


Finished 11/23
Finished 12/23


/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/sklearn/cluster/_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(
/Users/kostadindev/.conda/envs/constituency_parse/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__

Finished 13/23


In [111]:
# df['answers'].apply(lambda x:len(x))
# pd_print_entry(df, 21)
print(len(entries[0]))
df = pd.DataFrame(entries, columns =['sentence', 'questions', 'answers','removed', 'phrases'])
df[['sentence', 'questions', 'answers', 'phrases']]


5


,sentence,questions,answers,phrases
0,In the case current file handle represents an ...,[What current file handle represents an object...,[<np>the case current file handle represents a...,"{'PP': [(In, the, case, current, file, handle,..."
1,If the file does not exist at the server and c...,[What if the file does not exist at the server...,[<np>the file<np> <vp>does not exist at the se...,"{'SBAR': [(If, the, file, does, not, exist, at..."
2,The client has the choice of three creation me...,[],[],"{'NP': [(The, client), (the, choice, of, three..."
3,If the current filehandle is a named attribute...,[What if the current filehandle is a named att...,[<np>the current filehandle<np> <vp>is a named...,"{'SBAR': [(If, the, current, filehandle, is, a..."
4,"For this type of create, createattrs specifies...",[],[],"{'PP': [(For, this, type, of, create), (of, cr..."
5,The set of attributes may include any writable...,[],[],"{'NP': [(The, set, of, attributes), (The, set)..."
6,When an UNCHECKED create encounters an existin...,[What when an unchecked create encounters an e...,[<np>an UNCHECKED create<np> <vp>encounters an...,"{'SBAR': [(When, an, UNCHECKED, create, encoun..."
7,"If GUARDED4 is specified, the server checks fo...",[What if guarded4 is specified?],"[<np>GUARDED4<np> <vp>is specified<vp>, <np>GU...","{'SBAR': [(If, GUARDED4, is, specified)], 'S':..."
8,"If a duplicate exists, an error of NFS4ERR_EXI...",[What if a duplicate exists?],"[<np>a duplicate<np> <vp>exists<vp>, <np>a dup...","{'SBAR': [(If, a, duplicate, exists)], 'S': [(..."
9,"If the object does not exist, the server creat...",[What if the object does not exist?],"[<np>the object<np> <vp>does not exist<vp>, <n...","{'SBAR': [(If, the, object, does, not, exist)]..."


In [109]:
print(len(entries), len(entries[0]))

23 5


In [113]:
pd_print_entry(df, 21)
# df['answers'].apply(lambda x: len(x))

Sentence: 
If the component provided to OPEN resolves to something other than a regular file (or a named attribute), an error will be returned to the client.
Questions: 
['What if the component provided to open resolves to something other than a regular file (or a named attribute)?']
Answers
<np>the component provided to OPEN<np> <vp>provided to OPEN<vp>
<np>the component provided to OPEN<np> <vp>resolves to something other than a regular file (or a named attribute)<vp>
<np>the component provided to OPEN<np> <vp>will be returned to the client<vp>
<np>the component provided to OPEN<np> <vp>be returned to the client<vp>
<np>the component provided to OPEN<np> <vp>returned to the client<vp>
<np>the component<np> <vp>provided to OPEN<vp>
<np>the component<np> <vp>resolves to something other than a regular file (or a named attribute)<vp>
<np>the component<np> <vp>will be returned to the client<vp>
<np>the component<np> <vp>be returned to the client<vp>
<np>the component<np> <vp>returned to t